# MEET THE F#

## Quick introduction to F#

In [ ]:
open System

let wordCount (text: string) =
    let words = text.Split [|' '|]
    let wordSet = Set.ofArray words
    let nWords = words.Length
    let nDups = words.Length - wordSet.Count
    (nWords, nDups)

let showWordCount text =
    let nWords, nDups = wordCount text
    printfn "--> %d words in the text" nWords
    printfn "--> %d duplicate words" nDups

showWordCount "Lorem ipsum lorem not-ipsum and Lorem ipsum again..."

### 1

In [23]:
let calculateSomething (calculate: float -> float -> int) : int =
    calculate 3.1 0.9

let x = calculateSomething (fun x y -> int (x + y))

x

4

### 2

In [ ]:
let giveMeTuple (tuple: int * float) =
    let x, y = tuple
    x + int y

giveMeTuple (1, 4.5)

### 3

In [ ]:
let giveMeGenericTuple<'a, 'b> (tuple: 'a * 'b) =
    let x, y = tuple
    (x.ToString(), y.ToString())

giveMeGenericTuple (3, "Hi!")

### 4

In [ ]:
let inline ``give me generic tuple with constraints``<'a, 'b when 'b : (member Length : int)> (tuple: 'a * 'b) =
    let x, y = tuple
    (x.ToString(), y.Length)

``give me generic tuple with constraints`` (3, "Hi!")

### 5

In [ ]:
let ``amIafunction?`` =
    let x = 3 + 2
    let y = 3 * 2
    x * y

``amIafunction?``

### 6

In [ ]:
let ``amIafunction?`` () =
    let x = 3 + 2
    let y = 3 * 2
    x * y

``amIafunction?``

### Loops and recursion

#### For loop

In [ ]:
for i in 1..10 do
    printfn "F#: %d" i

#### While loop

In [ ]:
let mutable i = 1
while i <= 10 do
    printfn "F#: %d" i
    i <- i + 1

#### For-each loop

In [ ]:
let numbers = [1; 2; 3; 4; 5]
for number in numbers do
    printfn "F#: %d" number

#### Recursion (and also if statement)

In [ ]:
let rec loop i max =
    if i <= max then
        printfn "F#: %d" i
        loop (i + 1) max
    elif i > max then
        printfn "This is the end..."
    else
        printfn "How did you end up here?"

loop 3 7

## Let's get more functional!

In [ ]:
let squareF x = x * x

let sumOfSquaresF n =
   [1.0 .. n] |> List.map squareF |> List.sum

sumOfSquaresF 100.0

### Pattern matching

In [ ]:
let whatAmI data =
    match data with
    | [] -> printfn "Empty list!"
    | firstElement::otherElements -> 
        printfn "first element = %A, other elements = %A" firstElement otherElements

let whatRangeAmIIn x =
    match x with
    | x when x > 10 && x < 20 -> printfn "I am between 10 and 20"
    | _ -> printfn "I don't know, sorry :("

whatAmI []
whatAmI [ 1; 2; 3; 4; 5 ]

printfn ""

whatRangeAmIIn 19
whatRangeAmIIn 9

## Four Key Concepts

In [ ]:
let square x = x * x

// functions as values
let squareclone = square
let result = [1..10] |> List.map squareclone

// functions taking other functions as parameters
let execFunction aFunc aParam = aFunc aParam
let result2 = execFunction square 12

But C# has first-class functions too, so what’s so special about functional programming?

The short answer is that the function-oriented nature of F# infiltrates every part of the language and type system in a way that it does not in C#, so that things that are awkward or clumsy in C# are very elegant in F#.

### Expressions rather than statements (see vs C#)

- What initial value should result be set to?
- What if I forget to assign to the result variable?
- What is the value of the result variable in the “else” case?

- The result variable is declared at the same time that it is assigned. No variables have to be set up “outside” the expression and there is no worry about what initial value they should be set to.
- The “else” is explicitly handled. There is no chance of forgetting to do an assignment in one of the branches.
- It is not possible to forget to assign result, because then the variable would not even exist!

### Algebraic Types

It's a way of building types by combining existing types in two different ways:
- First, a combination of values, each picked from a set of types. These are called “product” types.
- Or, alternately, as a disjoint union representing a choice between a set of types. These are called “sum” types.

In [ ]:
//declare it
type IntAndBool = {intPart: int; boolPart: bool}

//use it
let x = {intPart=1; boolPart=false}

In [ ]:
//declare it
type IntOrBool =
  | IntChoice of int
  | BoolChoice of bool

//use it
let y = IntChoice 42
let z = BoolChoice true

y, z

## Pattern matching (but now with union types)

In [ ]:
type Shape =        // define a "union" of alternative structures
    | Circle of radius:int
    | Rectangle of height:int * width:int
    | Point of x:int * y:int
    | Polygon of pointList:(int * int) list

let draw shape =    // define a function "draw" with a shape param
  match shape with
  | Circle radius ->
      printfn "The circle has a radius of %d" radius
  | Rectangle (height,width) ->
      printfn "The rectangle is %d high by %d wide" height width
  | Polygon points ->
      printfn "The polygon is made of these points %A" points
  | _ -> printfn "I don't recognize this shape"

let circle = Circle(10)
let rect = Rectangle(4,5)
let point = Point(2,3)
let polygon = Polygon([(1,1); (2,2); (3,3)])

[circle; rect; polygon; point] |> List.iter draw

You might be wondering if this kind of pattern matching is a good idea? In an object-oriented design, checking for a particular class is an anti-pattern because you should only care about behavior, not about the class that implements it.

But in a pure functional design there are no objects and no behavior. There are functions and there are “dumb” data types. Data types do not have any behavior associated with them, and functions do not contain data – they just transform data types into other data types.

## Be not afraid of creating types!

In [ ]:
open System

// some "record" types
type Person = {FirstName:string; LastName:string; Dob:DateTime}
type Coord = {Lat:float; Long:float}

// some "union" (choice) types
type TimePeriod = Hour | Day | Week | Year
type Temperature = C of int | F of int
type Appointment =
  OneTime of DateTime | Recurring of DateTime list

In [ ]:
type CustomerType  = Prospect | Active | Inactive

// override equality and deny comparison
[<CustomEquality; NoComparison>]
type CustomerAccount ={
  CustomerAccountId: int;
  CustomerType: CustomerType;
  ContactInfo: string;
}

  override this.Equals(other) =
    match other with
    | :? CustomerAccount as otherCust ->
      (this.CustomerAccountId = otherCust.CustomerAccountId)
    | _ -> false

  override this.GetHashCode() = hash this.CustomerAccountId

## Extract boilerplate code

In [ ]:
let product n =
    let initialValue = 1
    [1..n] |> List.fold (fun productSoFar x -> productSoFar * x) initialValue

let sumOfOdds n =
    let initialValue = 0
    let action sumSoFar x = if x%2=0 then sumSoFar else sumSoFar+x
    [1..n] |> List.fold action initialValue

let alternatingSum n =
    let initialValue = (true,0)
    let action (isNeg,sumSoFar) x = if isNeg then (false,sumSoFar-x)
                                             else (true ,sumSoFar+x)
    [1..n] |> List.fold action initialValue |> snd

product 10

## Compose (F#'s fluent interface, strategy pattern, decorator pattern)

In [ ]:
// building blocks
let add2 x = x + 2
let mult3 x = x * 3
let square x = x * x

// test
[1..10] |> List.map add2 |> printfn "%A"
[1..10] |> List.map mult3 |> printfn "%A"
[1..10] |> List.map square |> printfn "%A"

// new composed functions
let add2ThenMult3 = add2 >> mult3     // let add2ThenMult3 x = mult3 (add2 x)
let mult3ThenSquare = mult3 >> square // let mult3ThenSquare x = square (mult3 x)

In [ ]:
add2ThenMult3 5 |> printfn "%A"
mult3ThenSquare 5 |> printfn "%A"
[1..10] |> List.map add2ThenMult3 |> printfn "%A"
[1..10] |> List.map mult3ThenSquare |> printfn "%A"

### Add logging behavior

In [ ]:
// helper functions;
let logMsg msg x = printf "%s%i" msg x; x     //without linefeed
let logMsgN msg x = printfn "%s%i" msg x; x   //with linefeed

// new composed function with new improved logging! (ugly name but easy to use!)
let mult3ThenSquareLogged =
   logMsg "before="
   >> mult3
   >> logMsg " after mult3="
   >> square
   >> logMsgN " result="

// test
mult3ThenSquareLogged 5
printfn "With list:"
[1..10] |> List.map mult3ThenSquareLogged //apply to a whole list

In [ ]:
let listOfFunctions = [
   mult3;
   square;
   add2;
   logMsgN "result=";
   ]

// compose all functions in the list into a single one
let allFunctions = List.reduce (>>) listOfFunctions

//test
allFunctions 5

## Create new operators

In [ ]:
let (|+|) a b = abs(a + b)
let (|-|) a b = abs(a - b)

-10 |+| (-3) |> printfn "%A"
-10 |-| (-3) |> printfn "%A"

## The strategy pattern in F#!

In [ ]:
type Animal(noiseMakingStrategy) =
   member this.MakeNoise =
      noiseMakingStrategy() |> printfn "Making noise %s"

// now create a cat
let meowing() = "Meow"
let cat = Animal(meowing)
cat.MakeNoise

## Partial application

In [ ]:
let funcWithLotsOfParams x y z a b c = x + y + z - (a + b + c)

let smallerFunc = funcWithLotsOfParams 5 5 5

// Last params (a, b, c) are automatically generated for smallerFunc. This is why in F#
// good approach is to make last parameters the most changing ones in given case.
smallerFunc 1 1 1

## Avoiding nulls with Option type!

In [ ]:
let getFileInfo filePath =
   let fi = new System.IO.FileInfo(filePath)
   if fi.Exists then Some(fi) else None

let goodFileName = "good.txt"
let badFileName = "bad.txt"

let goodFileInfo = getFileInfo goodFileName // Some(fileinfo)
let badFileInfo = getFileInfo badFileName   // None

match goodFileInfo with
  | Some fileInfo ->
      printfn "the file %s exists" fileInfo.FullName
  | None ->
      printfn "the file doesn't exist"

match badFileInfo with
  | Some fileInfo ->
      printfn "the file %s exists" fileInfo.FullName
  | None ->
      printfn "the file doesn't exist"

## Exhaustive pattern matching for edge cases

In [ ]:
let rec movingAverages list =
    match list with
    // if input is empty, return an empty list
    | [] -> []
    // otherwise process pairs of items from the input
    | x::y::rest ->
        let avg = (x+y)/2.0
        //build the result by recursing the rest of the list
        avg :: movingAverages (y::rest)
    | [_] -> []

// test
movingAverages [1.0]
movingAverages [1.0; 2.0]
movingAverages [1.0; 2.0; 3.0]

## Units of Measure

In [ ]:
// Mass, grams.
[<Measure>] type g
// Mass, kilograms.
[<Measure>] type kg
// Weight, pounds.
[<Measure>] type lb

// Distance, meters.
[<Measure>] type m
// Distance, cm
[<Measure>] type cm

// Distance, inches.
[<Measure>] type inch
// Distance, feet
[<Measure>] type ft

// Time, seconds.
[<Measure>] type s

// Force, Newtons.
[<Measure>] type N = kg m / s^2

// Pressure, bar.
[<Measure>] type bar
// Pressure, Pascals
[<Measure>] type Pa = N / m^2

// Volume, milliliters.
[<Measure>] type ml
// Volume, liters.
[<Measure>] type L

// Define conversion constants.
let gramsPerKilogram : float<g kg^-1> = 1000.0<g/kg>
let cmPerMeter : float<cm/m> = 100.0<cm/m>
let cmPerInch : float<cm/inch> = 2.54<cm/inch>

let mlPerCubicCentimeter : float<ml/cm^3> = 1.0<ml/cm^3>
let mlPerLiter : float<ml/L> = 1000.0<ml/L>

// Define conversion functions.
let convertGramsToKilograms (x : float<g>) = x / gramsPerKilogram
let convertCentimetersToInches (x : float<cm>) = x / cmPerInch

## Classes and interfaces

In [ ]:
// interface
type IEnumerator<'a> =
    abstract member Current : 'a
    abstract MoveNext : unit -> bool

// abstract base class with virtual methods
[<AbstractClass>]
type Shape() =
    //readonly properties
    abstract member Width : int with get
    abstract member Height : int with get
    //non-virtual method
    member this.BoundingArea = this.Height * this.Width
    //virtual method with base implementation
    abstract member Print : unit -> unit
    default this.Print () = printfn "I'm a shape"

// concrete class that inherits from base class and overrides
type Rectangle(x:int, y:int) =
    inherit Shape()
    override this.Width = x
    override this.Height = y
    override this.Print ()  = printfn "I'm a Rectangle"

//test
let r = Rectangle(2,3)
printfn "The width is %i" r.Width
printfn "The area is %i" r.BoundingArea
r.Print()

## Example project in F#

https://github.com/Pawleshhh/PracticalAstronomy

## Not liking F# but still want to use functional principles in C#?

https://github.com/Pawleshhh/FunctionalCSharp